In [ ]:
""" Hyperparameter Optimization 
- Grid Search: exhaustively considers all parameter combinations
    Halving strategies exist
    
- Random Search: sample a given number of candidates from a parameter space with a specified distribution

- Differential Evolution:
    Documentation:
    Library:
    
- DE from Scratch:

- PSO
    
- Evolutionary Powell's Method:
    Documentation: https://e2eml.school/evopowell.html
    Library: https://gitlab.com/brohrer/ponderosa
    
    
    
Using accuracy is useless if unbalanced, need to add weights to imageprocess class if the data is not balanced
Need to add function to HPO that will save DF to a csv, returns path of file

Use Parallelization
Use CV
"""

In [ ]:
import os
import numpy as np
import random
import pandas as pd
import seaborn as sns

# THIS IS NOT FINISHED
def DE_optimizer(eval_fxn(), bounds, args=(), pop_size=10, max_iterations=5, F=.5, cr=.9, drop=.5 )

    # define mutation operation
    def mutate(x, F):
        dim = len(x[0])
        mutation = [x[1][i]-x[2][i] for i in range(dim)]
        mutation = [mutation[i]*F for i in range(dim)]
        mutation = [x[0][i]+ mutation[i] for i in range(dim)]
        mutation = [int(mutation[i]) for i in range(dim)]
        return mutation

    # define boundary check operation
    def clip_bounds(x, bounds):
        clipped = [np.clip(x[i], 0 , bounds[i]) for i in range(len(bounds))]
        return clipped

    # define crossover operation creating solution vector by binomial crossover
    def crossover(x, y, cr):
        dim = len(x); p = np.random.uniform(size=dim)    
        return [x[i] if p[i] < cr else y[i] for i in range(dim)]

    def convert(x, key):
        return [key[i][x[i]] for i in range(len(x))] 
    
    # Generate and Evaluate Initial population of solutions
    population = [random.choices(y[x], k=pop_size) for x in range(dim)] 
    population = list(map(list, zip(*population)))

    # Evaluate generated solutions
    for i in range(pop_size):
        solution = convert(population[i],y_key)
        model_params = Model_Params(solution)
        model_weights[i], model_errors[i], history = fit_model(model_params, img_obj)

        # Store the solution in the all_solutions dict
        all_models[str(solution)] = history


    # Drop the 5 worst performers from the population
    lowest = np.array(model_errors).argsort()[:int(drop*pop_size)]
    lowest = sorted(lowest, reverse=True)
    population = [population.pop(x) for x in lowest]
    model_errors = [model_errors.pop(x) for x in lowest]
    model_weights = [model_weights.pop(x) for x in lowest]
    pop_size = len(population)

    print(population)
    print(model_errors)
    print("Min Error:", min(model_errors))

    # main processing loop
    for g in range(max_gen):
        for i in range(pop_size):  # For each solution in population

            # Check if the solution has been evaluated
            while True:
                # Choose 3 solutions
                indices = np.arange(pop_size)  # [0, 1, 2, . . ]
                np.random.shuffle(indices)
                abc = indices[:3].tolist()

                # If current solution is in set, replace
                if i in abc: abc[abc.index(i)]= indices[3]

                # Selected population of 3 solutions
                abc = [population[abc[x]] for x in range(3)] 

                # Mutate + Fix the solution
                solution = mutate(abc, F)
                solution = clip_bounds(solution, bounds)

                # Crossover
                solution = crossover(solution, population[i],cr)

                # Check the solution is unique (this will not account for the solutions we already deleted... but it should)
                if str(convert(solution,y_key)) not in all_models: break

        # replace curr soln with new soln if new soln is better
        converted_solution = convert(solution,y_key)
        model_params = Model_Params(converted_solution)
        new_weights, new_error, history = fit_model(model_params, img_obj)
    
        # Store the solution in the all_solutions dict
        all_models[str(converted_solution)] = history

        if new_error < model_errors[i]:
            print('Solution Replaced')
            population[i] = solution
            model_errors[i] = new_error
            model_weights[i] = new_weights
            
            

        # after all popln items have been processed, 
        # find curr best soln
        best_idx = np.argmin(popln_errors)
        best_error = popln_errors[best_idx]
        if g % 10 == 0:
            print("Generation = %4d best error = %10.4f \
            best_soln = " % (g, best_error), end="")
            print(population[best_idx])

        # show final result
        best_idx = np.argmin(popln_errors)
        best_error = popln_errors[best_idx]
        print("\nFinal best error = %0.4f  best_soln = " % \
        best_error, end="")
        print(population[best_idx])



## Using  Grid Search for Hyperparameter Optimization

## Using  Grid Search with Halving for Hyperparameter Optimization:

## Using Random Search for Hyperparameter Optimization

## Using Differential Evolution for Hyperparameter Optimization: SciPy Implementation

In [ ]:
print("\nUsing Differential Evolution for Hyperparameter Optimization: SciPy Implementation")

# Solve for Optimal Parameters
solution = sd.differential_evolution(hpo_model.evaluate_model, hpo_model.bounds, args=(weights),
                                  strategy='best1bin', 
                                  maxiter=max_gen, popsize=pop_size, 
                                  mutation=(F, 1), recombination=cr )

# Convert Optimal Parameters to English...
solution=hpo_model.name_model(hpo_model.translate_floats(solution.x))
best_accuracy = hpo_model.all_models[str(solution)].val_accuracy
print(f'Optimal Parameters: {solution} \nValuation Accuracy: {best_accuracy*100:.2f}%')

# Print Metrics
hpo_model.model_metrics()

hpo_data['DE: SciPy'] = hpo_model.export_to_csv()


## Using Differential Evolution for Hyperparameter Optimization: DE from Scratch

## Using Powell's Method for Hyperparameter Optimization


## Method Comparison

In [ ]:
# Frequency Graphs
op = [x for x in y if len(y[x])>1]

# Frequency Graphs
for i, col in enumerate(op):
    plt.figure(i)
    sns.countplot(x=col, data=df)

# Violins
for i, col in enumerate(op):
    plt.figure(i+5)
    sns.violinplot(x=col, y="val_accuracy", data=df)